<a href="https://colab.research.google.com/github/febrianrachmad/TEST_AGRIAKU/blob/main/DATA_COBA_TEST_AGRIAKU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from google.colab import files

uploaded = files.upload()


Saving lecturer.csv to lecturer (2).csv
Saving student.csv to student (2).csv
Saving enrollment.csv to enrollment (1).csv
Saving schedule.csv to schedule (1).csv
Saving course.csv to course (2).csv
Saving course_attendance.csv to course_attendance (1).csv


In [33]:
import pandas as pd

enrollment_df = pd.read_csv('enrollment.csv')
student_df = pd.read_csv('student.csv')
lecturer_df = pd.read_csv('lecturer.csv')
schedule_df = pd.read_csv('schedule.csv')
course_df = pd.read_csv('course.csv')
course_attendance_df = pd.read_csv('course_attendance.csv')


In [34]:
enrollment_df['ENROLL_DT'] = pd.to_datetime(enrollment_df['ENROLL_DT'], format='%d-%b-%y')
schedule_df['START_DT'] = pd.to_datetime(schedule_df['START_DT'], format='%d-%b-%y')
schedule_df['END_DT'] = pd.to_datetime(schedule_df['END_DT'], format='%d-%b-%y')
course_attendance_df['ATTEND_DT'] = pd.to_datetime(course_attendance_df['ATTEND_DT'], format='%d-%b-%y')

In [35]:
merged_df = course_attendance_df.merge(enrollment_df, left_on=['STUDENT_ID', 'SCHEDULE_ID'], right_on=['STUDENT_ID', 'SCHEDULE_ID'])
merged_df = merged_df.merge(schedule_df, left_on='SCHEDULE_ID', right_on='ID')
merged_df = merged_df.merge(course_df, left_on='COURSE_ID', right_on='ID', suffixes=('', '_course'))

In [36]:
merged_df['WEEK_ID'] = (merged_df['ATTEND_DT'] - merged_df['START_DT']).dt.days // 7 + 1

In [37]:
merged_df = merged_df[(merged_df['WEEK_ID'] >= 1) & (merged_df['WEEK_ID'] <= 13)]

In [38]:
total_students_per_course = enrollment_df.groupby(['SCHEDULE_ID', 'SEMESTER'])['STUDENT_ID'].nunique().reset_index()
total_students_per_course = total_students_per_course.rename(columns={'STUDENT_ID': 'TOTAL_STUDENTS'})

In [39]:
merged_df = merged_df.merge(total_students_per_course, left_on=['SCHEDULE_ID', 'SEMESTER'], right_on=['SCHEDULE_ID', 'SEMESTER'])

In [40]:
weekly_attendance = merged_df.groupby(['SEMESTER', 'WEEK_ID', 'COURSE_ID', 'NAME']).agg({
    'STUDENT_ID': 'nunique',
    'TOTAL_STUDENTS': 'first'
}).reset_index()

In [41]:
weekly_attendance['ATTENDANCE_PCT'] = (weekly_attendance['STUDENT_ID'] / weekly_attendance['TOTAL_STUDENTS']) * 100
weekly_attendance = weekly_attendance.rename(columns={
    'SEMESTER': 'SEMESTER_ID',
    'NAME': 'COURSE_NAME',
    'ATTENDANCE_PCT': 'ATTENDANCE_PCT (%)'
})

In [42]:
weekly_attendance = weekly_attendance[['SEMESTER_ID', 'WEEK_ID', 'COURSE_NAME', 'ATTENDANCE_PCT (%)']].sort_values(by=['SEMESTER_ID', 'WEEK_ID', 'COURSE_NAME'])

In [45]:
weekly_attendance.to_csv('weekly_attendance.csv', index=False)

In [51]:
files.download('weekly_attendance.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>